<a href="https://colab.research.google.com/github/kelinlin-star/MSSP6070/blob/main/Assignments/Participation_Activity_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import os
import requests
from bs4 import BeautifulSoup
import time

# Function to download images from a given Imgur search URL
def download_images(search_query, num_images=10):
    # Create a directory to store images
    if not os.path.exists(search_query):
        os.makedirs(search_query)

    # Create the search URL on Imgur
    url = f"https://imgur.com/search?q={search_query}"

    # Custom headers to mimic a browser request
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    # Send an HTTP request to get the content of the page
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to retrieve the page: {response.status_code}")
        return

    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all image tags in the page, which could be lazy-loaded
    img_tags = soup.find_all('img')

    # Limit the number of images to download (to prevent downloading too many)
    img_count = 0

    for img_tag in img_tags:
        # Get the image URL from the 'src' or 'data-src' attribute (for lazy-loaded images)
        img_url = img_tag.get('src') or img_tag.get('data-src')

        # Only consider images with a valid URL
        if img_url and img_url.startswith('https://'):
            # Get the file name from the image URL
            img_name = os.path.join(search_query, img_url.split('/')[-1])

            # Download the image
            try:
                img_data = requests.get(img_url).content
                with open(img_name, 'wb') as f:
                    f.write(img_data)
                    print(f"Downloaded {img_name}")
                    img_count += 1
                if img_count >= num_images:  # Stop after downloading the specified number of images
                    break
            except Exception as e:
                print(f"Failed to download {img_url}: {e}")

            # Wait a few seconds before making another request to avoid rate limiting
            time.sleep(2)  # Sleep for 2 seconds

# Main entry point
if __name__ == '__main__':
    search_query = input("Enter the category of images you want to download (e.g., cats, nature): ")
    num_images = int(input("How many images would you like to download? "))
    download_images(search_query, num_images)


Enter the category of images you want to download (e.g., cats, nature): cats
How many images would you like to download? 2
